In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.neighbors import NearestCentroid

In [ ]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

listStopword = set(stopwords.words('english'))
def text_preprocessing(text):
    new_word = list()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r'\s+', ' ', text)
    text = text.lower()
    text = nltk.tokenize.word_tokenize(text)
    for word in text:
      if word not in listStopword:
            new_word.append(word)
    return new_word
    

In [ ]:
from pandas import DataFrame
# import data berita
data_true = pd.read_csv('../input/fake-and-real-news-dataset/True.csv');
data_fake = pd.read_csv('../input/fake-and-real-news-dataset/Fake.csv');

# ambil konten berita dan valuenya
berita = []

# mapping data
for index, x in enumerate(data_fake['text']):
  temp = text_preprocessing(x)
  berita.append([temp, 0])
  if(index==2499):
    break

for index, x in enumerate(data_true['text']):
  temp = text_preprocessing(x)
  berita.append([temp, 1])
  if(index==2499):
    break

df = DataFrame(berita, columns=['token_berita','value'])
df.head()


In [ ]:

def calc_TF(document):
    # Counts the number of times the word appears in review
    TF_dict = {}
    for term in document:
        if term in TF_dict:
            TF_dict[term] += 1
        else:
            TF_dict[term] = 1
    # Computes tf for each word
    for term in TF_dict:
        TF_dict[term] = TF_dict[term] / len(document)
    return TF_dict

df["TF_dict"] = df['token_berita'].apply(calc_TF)
df["TF_dict"].head()


In [ ]:
# Check TF result
index = 499

print('%20s' % "term", "\t", "TF\n")
for key in df["TF_dict"][index]:
    print('%20s' % key, "\t", df["TF_dict"][index][key])

In [ ]:

def calc_DF(tfDict):
    count_DF = {}
    # Run through each document's tf dictionary and increment countDict's (term, doc) pair
    for document in tfDict:
        for term in document:
            if term in count_DF:
                count_DF[term] += 1
            else:
                count_DF[term] = 1
    return count_DF

DF = calc_DF(df["TF_dict"])
DF

In [ ]:
n_document = len(df)

def calc_IDF(__n_document, __DF):
    IDF_Dict = {}
    for term in __DF:
        IDF_Dict[term] = np.log(__n_document / (__DF[term] + 1))
    return IDF_Dict
  
#Stores the idf dictionary
IDF = calc_IDF(n_document, DF)
with open('IDF.csv', 'w') as f:
    for key in IDF.keys():
        f.write("%s,%s\n"%(key,IDF[key]))
IDF


In [ ]:
#calc TF-IDF
def calc_TF_IDF(TF):
    TF_IDF_Dict = {}
    #For each word in the review, we multiply its tf and its idf.
    for key in TF:
        TF_IDF_Dict[key] = TF[key] * IDF[key]
    return TF_IDF_Dict

#Stores the TF-IDF Series
df["TF-IDF_dict"] = df["TF_dict"].apply(calc_TF_IDF)

In [ ]:
index = 4

print('%20s' % "term", "\t", '%10s' % "TF", "\t", '%20s' % "TF-IDF\n")
for key in df["TF-IDF_dict"][index]:
    print('%20s' % key, "\t", df["TF_dict"][index][key] ,"\t" , df["TF-IDF_dict"][index][key])


In [ ]:
hasil = []
for index, key in enumerate(df['value']):
  hasil.append(key)
tf_idf = []
for index, key in enumerate(df['TF-IDF_dict']):
  temp=[]
  for term in IDF:
    flag = 0
    for value in key:
      if(term == value):
        temp.append(key[value])
        flag = 1
    if(flag==0):
      temp.append(0)    
  tf_idf.append(temp)

In [ ]:
# y = np.array(hasil)
# x = np.array(tf_idf)
clf = NearestCentroid()
clf.fit(tf_idf, hasil)

In [ ]:
from joblib import dump, load
dump(clf, 'rocchio_model.joblib') 

In [ ]:
temp=[]
for term in IDF:
  flag = 0
  for value in df['TF-IDF_dict'][1]:
    if(term == value):
      temp.append(df['TF-IDF_dict'][1][value])
      flag = 1
  if(flag==0):
    temp.append(0)    
temp2 = [temp]
clf2 = load('rocchio_model.joblib') 
# test = df['TF-IDF_dict'][0].values()
# test2 = list(test)
# test3 = [test2]
# print(test3)
print(clf2.predict(temp2))